## Spark Streaming

-   Procesamiento escalable, *high-throughput* y tolerante a fallos de flujos de datos

<img src="http://localhost:8085/figs/streaming-flow.png" alt="Flujo de Spark Streaming" style="width: 800px;"/>

-   Entrada desde muchas fuentes: Kafka, Flume, Twitter, ZeroMQ, Kinesis o sockets TCP

### Arquitectura de Spark Streaming

Abstracción principal: DStream (`discretized stream`).

-   Representa un flujo continuo de datos

![dstreams](http://localhost:8085/figs/dstreams.png)

Arquitectura *micro-batch*

-   Los datos recibidos se agrupan en batches
-   Los batches se crean a intervalos regulares (batch interval)
-   Cada batch forma un RDD, que es procesado por Spark
-   Adicionalmente: transformaciones con estado mediante
    -   Operaciones con ventanas
    -   Tracking del estado por cada clave

Página de Spark Streaming: <https://spark.apache.org/streaming/>
Documentación principal (de la última versión): <https://spark.apache.org/docs/latest/streaming-programming-guide.html>

### Spark Streaming: ejemplo de WordCount en red y sin estado

Para ejecutar el ejemplo:

   1. Ve al terminal desde el que iniciaste la máquina virtual con `vagrant up` y ejecuta la orden `vagrant ssh`
   2. Una vez en el terminal de la máquina virtual, usa netcat como un servidor en el puerto 9999

    `$ nc -lk 9999`

   2. Ejecuta el código PySpark que viene a continuación 

   3. Escribe líneas en el terminal del netcat, que serán recogidas y procesadas por el script
    - Escribe palabras repetidas, para comprobar que las cuenta bien

In [2]:
from pyspark.streaming import StreamingContext
from operator import add

sc.setLogLevel("WARN")

# Contexto Streaming con un batch interval de 5 s
ssc = StreamingContext(sc, 5)

# DStream que conecta a localhost:9999
lines = ssc.socketTextStream("localhost", 9999)

# Ejecuta un WordCount
counts = lines.flatMap(lambda line: line.split(" "))\
              .map(lambda word: (word, 1))\
              .reduceByKey(add)
              
counts.pprint()

ssc.start() # Inicia la computacion
ssc.awaitTerminationOrTimeout(60) # Espera a que termine (acaba en 60 segundos)

### Spark Streaming: ejemplo de WordCount en red con estado

Repite los pasos anteriores, ejecutando el siguiente código

 - Comprueba que el número de palabras se acumula entre accesos

In [4]:
from pyspark.streaming import StreamingContext
from operator import add

sc.setLogLevel("WARN")

# Contexto Streaming con un batch interval de 5 s
ssc = StreamingContext(sc, 5)

# DStream que conecta a localhost:9999
lines = ssc.socketTextStream("localhost", 9999)

ssc.checkpoint("/tmp/cpdir") # Activa checkpoint

def updateFunc(new_values, last_sum):
    return sum(new_values) + (last_sum or 0)

counts = lines.flatMap(lambda line: line.split(" "))\
              .map(lambda word: (word, 1))\
              .updateStateByKey(updateFunc)

counts.pprint()

ssc.start() # Inicia la computacion
ssc.awaitTerminationOrTimeout(60) # Espera a que termine (acaba en 60 segundos)